# Eigenvalues

## Eigenvalue decomposition

When it exists, the eigenvalue decomposition (EVD) expresses a square matrix $A$ as a change of basis (in both source and image spaces) to a  diagonal matrix: $A=XDX^{-1}$. There are two major syntaxes. If you call `eig`, you get the complete EVD. 

In [2]:
A = [
    16     2     3    13
     5    11    10     8
     9     7     6    12
     4    14    15     1

];
[X,D] = eig(A);
lam = diag(D)
X

lam =
   34.0000
    8.9443
   -8.9443
   -0.0000
X =
   -0.5000   -0.8236    0.3764   -0.2236
   -0.5000    0.4236    0.0236   -0.6708
   -0.5000    0.0236    0.4236    0.6708
   -0.5000    0.3764   -0.8236    0.2236


Technically, the output is guaranteed only to satisfy $AX=XD$, which holds for defective as well as diagonalizable matrices. (But defectiveness is a "probability zero" event and is not robust to floating point perturbations.)

In [4]:
A = 5*eye(4) + diag([1;1;1],1);   % 4x4 Jordan block
[X,D] = eig(A);
lam = diag(D)
rank(X)

lam =
     5
     5
     5
     5
ans =
     1


In [5]:
residual = norm(A*X-X*D)

residual =
   1.5384e-15


You can request the eigenvalues alone if you give just a single output to `eig`. This can be significantly faster than the full EVD.

## Schur factorization

The EVD has some serious drawbacks as a target for numerical computation. First, it does not exist for every square matrix. And second, if it exists but the condition number of the eigenvalue matrix is large, it will be very sensitive to roundoff. A stable alternative is the **Schur factorization** $A=QTQ^*$, in which $Q$ is unitary and $T$ is upper triangular.

In [7]:
A = [
    17    24     1     8    15
    23     5     7    14    16
     4     6    13    20    22
    10    12    19    21     3
    11    18    25     2     9
];

[Q,T] = schur(A);
T

T =
   65.0000    0.0000   -0.0000    0.0000   -0.0000
         0  -21.2768   -2.5888    2.1871   -3.4893
         0         0  -13.1263   -3.3845   -2.8239
         0         0         0   21.2768    2.6287
         0         0         0         0   13.1263


The Schur form always exists, it still reveals the eigenvalues of $A$, and it involves only stable unitary transformations. 

There is one quirk. If $A$ is real but has complex eigenvalues, there is an alternate form in which $T$ is "quasitriangular." Each real eigenvalue appears along the diagonal of $T$, but complex conjugate pairs are represented as $2\times 2$ real blocks along the diagonal. This allows the entire factorizaton to be done in real arithmetic, which can improve speed. 

In [8]:
A = rot90(A); 
eig(A)

ans =
  65.0000 + 0.0000i
   0.0000 +21.2768i
   0.0000 -21.2768i
   0.0000 +13.1263i
   0.0000 -13.1263i


In [9]:
[Q,T] = schur(A);
T

T =
   65.0000   -0.0000    0.0000   -0.0000   -0.0000
         0    0.0000   20.9795    6.8026    0.0000
         0  -21.5782    0.0000   -0.0000   -1.7035
         0         0         0    0.0000   13.4714
         0         0         0  -12.7900    0.0000


In [10]:
eig(T(2:3,2:3))

ans =
   0.0000 +21.2768i
   0.0000 -21.2768i


## Hessenbergification

Eigenvalue algorithms are iterative. The iterative steps can be performed a lot faster if the matrix has structural zeros. Hence before iteration begins, an orthogonal similarity transformation is used to get an **upper Hessenberg** matrix with the same eigenvalues.

In [11]:
H = hess(A)

H =
   15.0000  -12.2967  -25.7440    3.8568    1.0776
  -30.4959   34.5430   16.0572   -7.0391   -1.1312
         0   33.2719   17.3227    3.9749   -2.2465
         0         0  -10.6355   -0.0167  -17.4078
         0         0         0   13.8061   -1.8490


In [12]:
eig(H)

ans =
  65.0000 + 0.0000i
   0.0000 +21.2768i
   0.0000 -21.2768i
   0.0000 +13.1263i
   0.0000 -13.1263i


Note that $A=QHQ^*$ is "nearly" a Schur factorization. All that remains to be done is to make the subdiagonal zero. 

If $A$ is hermitian, then since the transformation preserves symmetry the resulting $H$ is actually hermitian and tridiagonal. This leads to even greater speedup in eigenvalue computations; for this and other reasons, one might even consider the hermitian and nonhermitian problems to be of distinct species.

In [13]:
A = A+A';
hess(A)

ans =
    0.1631    3.2854         0         0         0
    3.2854    4.8747    1.5240         0         0
         0    1.5240    0.4987   -7.5820         0
         0         0   -7.5820  102.4634  -54.0925
         0         0         0  -54.0925   22.0000


## Outer product interpretation

We can view the diagonalization $A=XDX^{-1}$ as $A=XDY^*$, where $Y^*X=I$. The rows of $Y^*$ are known as **left eigenvectors** of $A$.

In [14]:
A = [
    16     2     3    13
     5    11    10     8
     9     7     6    12
     4    14    15     1

];
[X,D] = eig(A);
Y = inv(X')

Y =
   -0.5000   -0.9340   -0.1840   -0.2236
   -0.5000    0.6840    0.4340   -0.6708
   -0.5000    0.4340    0.6840    0.6708
   -0.5000   -0.1840   -0.9340    0.2236


In [18]:
Y(:,1)'*A
D(1,1)*Y(:,1)' 

ans =
  -17.0000  -17.0000  -17.0000  -17.0000
ans =
  -17.0000  -17.0000  -17.0000  -17.0000


They are also the (right) eigenvectors of $A^*$. (If $A^*=A$, then of course $X=Y$ is orthogonal.)

In [20]:
A'*Y(:,1)
D(1,1)*Y(:,1)

ans =
  -17.0000
  -17.0000
  -17.0000
  -17.0000
ans =
  -17.0000
  -17.0000
  -17.0000
  -17.0000


Hence we can write $A=\sum \lambda_k x_k y_k^*$. While this is interesting, so far I don't know if it's very useful to us. 